# Import packages

In [1]:
import numpy as np; print("Numpy version: ",np.__version__)
import scipy as sc; print("Scipy version: ",sc.__version__)
import scipy.spatial as sps
from scipy.linalg import svd

Numpy version:  1.16.4
Scipy version:  1.2.1


In [2]:
# Set number of decimal places to 3
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

# Functions

In [3]:
# Define function to calculate the covariances

def covar ( t, d, r ):
    h = d / r
    if t == 1: #Spherical
        c = 1 - h * (1.5 - 0.5 * h**2)
        c[h > 1] = 0
    elif t == 2: #Exponential
        c = np.exp( -3 * h )
    elif t == 3: #Gaussian
        c = np.exp( -3 * h**2 )
    return c

## Data

In [31]:
""" This piece of code is used to generate correlated data given a correlation matrix. We used it to generate the vector of data
in the small example section"""

# Correlation matrix
corr_mat= np.array([[1.0, 0.62],
                    [0.62, 1.0]])

# Compute the (upper) Cholesky decomposition matrix
upper_chol = sc.linalg.cholesky(corr_mat)

# Generate series of normally distributed (Gaussian) numbers
rnd = np.random.normal(0.0, 1.0, size=(2, 2))

# Finally, compute the inner product of upper_chol and rnd
data = rnd @ upper_chol

## Problem setup

In [4]:
# Consider Z1 and Z2 from the example
z = np.array([[0.146, -0.264],[-1.207,  1.155]])
print("Vector of data values, z:")
print(z)

Vector of data values, z:
[[ 0.146 -0.264]
 [-1.207  1.155]]


In [5]:
# Define the LMC coefficients
A = np.array([[ 0.843 , 0.504 , 0.168 , 0.084], [ 0.347 , 0.347,  0.867 , 0.087]])

In [6]:
print("LMC squared coefficients explaining the contribution of the direct variogram structures")
print(A**2)

LMC squared coefficients explaining the contribution of the direct variogram structures
[[ 0.711  0.254  0.028  0.007]
 [ 0.120  0.120  0.752  0.008]]


In [7]:
print("LMC cross coefficients explaining the contribution of the cross variogram structures")
print(np.prod(A, axis=0))

LMC cross coefficients explaining the contribution of the cross variogram structures
[ 0.293  0.175  0.146  0.007]


In [8]:
print("Matrix of A coefficients:")
print(A,"\n")

print("Sum to unity check (check if the LMC coefficients add up to 1)")
print(np.sum(A**2,1),"\n")

print("A'A == correlation check (check if the LMC fits the correlation of 0.62)")
print(np.matmul(A,A.T))

Matrix of A coefficients:
[[ 0.843  0.504  0.168  0.084]
 [ 0.347  0.347  0.867  0.087]] 

Sum to unity check (check if the LMC coefficients add up to 1)
[ 1.000  1.000] 

A'A == correlation check (check if the LMC fits the correlation of 0.62)
[[ 1.000  0.620]
 [ 0.620  1.000]]


In [9]:
# Define some variables for the example
nvars = 2 # number of variables
nstruct = 4 # number of LMC structures
structtype = np.array([1,1,1,1]) # structure types (1 = spherical)
ranges = [2,4,7,10] # variogram ranges

In [10]:
# Define data locations
locs = np.array([[0,0],[0,1]])
print("Data locations")
print(locs)

Data locations
[[0 0]
 [0 1]]


## Covariances and cokriging

In [11]:
# Compute distance matrix and Setup block covariance matrix for all Y's

a = 0 * 3.14159265358979 / 180 # setting rotation to zero (no anisotropy)
rmat = np.asarray([[np.cos(a),-np.sin(a)],[np.sin(a),np.cos(a)]]) # define rotation matrix
p = locs.copy() # make a copy of data location
pm = np.mean(p) # centre calculations
q0 = np.matmul(p-pm,rmat)

nxy = locs.shape[0] # number of locations

C = [] # store covariances
for i in range(nstruct): # calculate covariances
    Q = q0.copy()
    Q[:,0] = Q[:,0] / ranges[i]
    Q[:,1] = Q[:,1] / ranges[i]
    d = sps.distance_matrix(Q,Q)
    c = covar(structtype[i],d,1)
    C.append(c)

In [12]:
print("Covariance matrices (by structure):")
for i in range(nstruct):
    print("Structure {}".format(i+1))
    print(C[i])

Covariance matrices (by structure):
Structure 1
[[ 1.000  0.312]
 [ 0.312  1.000]]
Structure 2
[[ 1.000  0.633]
 [ 0.633  1.000]]
Structure 3
[[ 1.000  0.787]
 [ 0.787  1.000]]
Structure 4
[[ 1.000  0.851]
 [ 0.851  1.000]]


In [13]:
print('Generating SigmaY (CY) and reshape A matrices (AY)')

CY = np.zeros([nstruct*nxy,nstruct*nxy])
AY = np.zeros([nvars*nxy,nstruct*nxy])
for i in range(nstruct):
    CY[i*nxy:(i+1)*nxy,i*nxy:(i+1)*nxy] = C[i]
    AY[0:nxy,i*nxy:(i+1)*nxy] = np.eye(nxy) * A[0,i]
    AY[nxy:2*nxy,i*nxy:(i+1)*nxy] = np.eye(nxy) * A[1,i]

print("CY = \n",CY)
print("\nAY = \n",AY)

Generating SigmaY (CY) and reshape A matrices (AY)
CY = 
 [[ 1.000  0.312  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.312  1.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  1.000  0.633  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.633  1.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  1.000  0.787  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.787  1.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  1.000  0.851]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.851  1.000]]

AY = 
 [[ 0.843  0.000  0.504  0.000  0.168  0.000  0.084  0.000]
 [ 0.000  0.843  0.000  0.504  0.000  0.168  0.000  0.084]
 [ 0.347  0.000  0.347  0.000  0.867  0.000  0.087  0.000]
 [ 0.000  0.347  0.000  0.347  0.000  0.867  0.000  0.087]]


In [14]:
print("Covariance matrix between Y and Z")
print("Cyz = CY @ AY.T")
Cyz = CY @ AY.T

print("Cyz = \n",Cyz)

print("\nCovariance matrix of Z")
print("CZ = AY @ CY @ AY.T")
CZ = AY @ CY @ AY.T

print("CZ = \n",CZ)

Covariance matrix between Y and Z
Cyz = CY @ AY.T
Cyz = 
 [[ 0.843  0.263  0.347  0.108]
 [ 0.263  0.843  0.108  0.347]
 [ 0.504  0.319  0.347  0.220]
 [ 0.319  0.504  0.220  0.347]
 [ 0.168  0.132  0.867  0.682]
 [ 0.132  0.168  0.682  0.867]
 [ 0.084  0.071  0.087  0.074]
 [ 0.071  0.084  0.074  0.087]]

Covariance matrix of Z
CZ = AY @ CY @ AY.T
CZ = 
 [[ 1.000  0.411  0.620  0.323]
 [ 0.411  1.000  0.323  0.620]
 [ 0.620  0.323  1.000  0.712]
 [ 0.323  0.620  0.712  1.000]]


In [15]:
print("Conditional mean solution (cokriging y given z)")
d = np.linalg.solve(CZ,np.reshape(z.T,[nvars*nxy,1]))
ybar = Cyz @ d
y0 = np.reshape(ybar,[nstruct,nxy]).T

print("y0 =\n",y0)

Conditional mean solution (cokriging y given z)
y0 =
 [[ 0.452 -0.343 -0.343 -0.053]
 [-1.705 -0.251  2.113  0.020]]


In [16]:
print("Checking the cokriging estimates to ensure numerical consistency of Zs")
print("Estimates matrix / reference matrix")

zt = np.matmul(A,y0.T).T
for i in range(2):
    print(zt[i,:],z[i,:])

Checking the cokriging estimates to ensure numerical consistency of Zs
Estimates matrix / reference matrix
[ 0.146 -0.264] [ 0.146 -0.264]
[-1.207  1.155] [-1.207  1.155]


## Singular value decomposition

In [17]:
u,s,v = svd(AY, full_matrices=True, lapack_driver='gesvd')

In [18]:
print("SVD matrices \n")
print("Matrix U =\n",u)
print("\nMatrix S =\n",s)
print("\nMatrix V =\n",v)

SVD matrices 

Matrix U =
 [[ 0.000 -0.707  0.707  0.000]
 [ 0.707  0.000  0.000 -0.707]
 [ 0.000 -0.707 -0.707  0.000]
 [ 0.707  0.000 -0.000  0.707]]

Matrix S =
 [ 1.273  1.273  0.616  0.616]

Matrix V =
 [[ 0.000  0.661  0.000  0.473  0.000  0.575  0.000  0.095]
 [-0.661 -0.000 -0.473 -0.000 -0.575 -0.000 -0.095 -0.000]
 [ 0.569 -0.000  0.180 -0.000 -0.802 -0.000 -0.003 -0.000]
 [-0.000 -0.569 -0.000 -0.180 -0.000  0.802 -0.000  0.003]
 [ 0.486  0.000 -0.860  0.000  0.152  0.000 -0.027  0.000]
 [ 0.000  0.486  0.000 -0.860  0.000  0.152  0.000 -0.027]
 [-0.048  0.000 -0.068  0.000 -0.054  0.000  0.995  0.000]
 [ 0.000 -0.048  0.000 -0.068  0.000 -0.054  0.000  0.995]]


In [23]:
# Orthogonal complement - row space of v
vorth = v[0:nvars*nxy,:]
vperp = v[nvars*nxy:,:]

print("Orthonormal basis, =\n",vorth)
print("\nOrthogonal complement, (row space of v) =\n",vperp)

# Computation of Lambda for computing covariance of X
Lambda = vorth @ (CY @ vorth.T)
Lambda = vorth.T @ np.linalg.solve(Lambda,vorth)
Lambda = np.eye(CY.shape[0]) - CY @ Lambda
Lambda = vperp @ Lambda

print("\nLambda = \n",Lambda)

Orthonormal basis, =
 [[ 0.000  0.661  0.000  0.473  0.000  0.575  0.000  0.095]
 [-0.661 -0.000 -0.473 -0.000 -0.575 -0.000 -0.095 -0.000]
 [ 0.569 -0.000  0.180 -0.000 -0.802 -0.000 -0.003 -0.000]
 [-0.000 -0.569 -0.000 -0.180 -0.000  0.802 -0.000  0.003]]

Orthogonal complement, (row space of v) =
 [[ 0.486  0.000 -0.860  0.000  0.152  0.000 -0.027  0.000]
 [ 0.000  0.486  0.000 -0.860  0.000  0.152  0.000 -0.027]
 [-0.048  0.000 -0.068  0.000 -0.054  0.000  0.995  0.000]
 [ 0.000 -0.048  0.000 -0.068  0.000 -0.054  0.000  0.995]]

Lambda = 
 [[ 0.430  0.144 -0.882  0.067  0.210 -0.078 -0.028  0.007]
 [ 0.144  0.430  0.067 -0.882 -0.078  0.210  0.007 -0.028]
 [-0.036 -0.030 -0.060 -0.019 -0.044 -0.011  0.997 -0.003]
 [-0.030 -0.036 -0.019 -0.060 -0.011 -0.044 -0.003  0.997]]


In [30]:
# Covariance of X and mean of X (should be zero)
CX = Lambda @ (CY @ vperp.T)
Xm = Lambda @ np.reshape(-y0.T,[nstruct*nxy,1])

print("Mean of Xm (should be zero):\n",np.mean(Xm),"\n")
print("Covariance of Xm (should be zero) \n",np.cov(Xm,rowvar=False),"\n")

Mean of Xm (should be zero):
 7.164407955784213e-16 

Covariance of Xm (should be zero) 
 6.680208382728652e-31 



In [33]:
print('SigmaX (CX):')
print(CX)

SigmaX (CX):


array([[ 0.976,  0.571,  0.004, -0.001],
       [ 0.571,  0.976, -0.001,  0.004],
       [ 0.004, -0.001,  0.999,  0.849],
       [-0.001,  0.004,  0.849,  0.999]])

## Simulation of the factors

In [34]:
print("Performing Cholesky decomposition of SigmaX")
# Cholesky of covariance of X for generating realizations
BigLower = np.linalg.cholesky(CX)

Performing Cholesky decomposition of SigmaX


In [35]:
print("Lower L matrix:")
print(BigLower)

Lower L matrix:
[[ 0.988  0.000  0.000  0.000]
 [ 0.578  0.801  0.000  0.000]
 [ 0.004 -0.005  0.999  0.000]
 [-0.001  0.006  0.849  0.527]]


In [75]:
"""Generate realizations of Y. This piece of code generates realizations of Y for any vector of random samples"""

# Consider the two vectors from the article
r = np.array([[-0.355,1.909,0.593,-1.076],
              [-0.260,0.129,-0.698,1.482]])

nreal = 2
numx = 2 # number of structures Y (4) - number of Z (2)
for ir in range(nreal):
    print("Realization #{}\n".format(ir+1))
    # r = np.random.normal(0,1,[numx*nxy,1])
    print("random vector r:")
    print(r[ir])
    
    x = BigLower @ r[ir]
    yi = np.reshape(vperp.T @ x,[nstruct,nxy]).T
    yt = yi + y0
    print("\n simulated y vector:")
    print(np.reshape(yt.T,[nstruct*nxy,1]))

    print("\n Computed z values:")
    zt = AY @ np.reshape(yt.T,[nstruct*nxy,1])
    zt = np.reshape(zt,[2,nxy]).T
    print(zt)
    print("\n Rreference z values:")
    print(z)
    print()

Realization #1

random vector r:
[-0.355  1.909  0.593 -1.076]

 simulated y vector:
[[ 0.254]
 [-1.058]
 [-0.081]
 [-1.386]
 [-0.427]
 [ 2.317]
 [ 0.536]
 [-0.066]]

 Computed z values:
[[ 0.146 -0.264]
 [-1.207  1.155]]

 Rreference z values:
[[ 0.146 -0.264]
 [-1.207  1.155]]

Realization #2

random vector r:
[-0.260  0.129 -0.698  1.482]

 simulated y vector:
[[ 0.361]
 [-1.737]
 [-0.075]
 [-0.224]
 [-0.344]
 [ 2.096]
 [-0.742]
 [ 0.210]]

 Computed z values:
[[ 0.146 -0.264]
 [-1.207  1.155]]

 Rreference z values:
[[ 0.146 -0.264]
 [-1.207  1.155]]

